In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.pipeline import Pipeline
from scikeras.wrappers import KerasClassifier
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping
import pickle

In [3]:
data=pd.read_csv('Churn_Modelling.csv')
data = data.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)

label_encoder_gender = LabelEncoder()
data['Gender'] = label_encoder_gender.fit_transform(data['Gender'])

onehot_encoder_geo = OneHotEncoder(handle_unknown='ignore')
geo_encoded = onehot_encoder_geo.fit_transform(data[['Geography']]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns=onehot_encoder_geo.get_feature_names_out(['Geography']))

data = pd.concat([data.drop('Geography', axis=1), geo_encoded_df], axis=1)

X = data.drop('Exited', axis=1)
y = data['Exited']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Save encoders and scaler for later use
with open('pickle_files/label_encoder_gender.pkl', 'wb') as file:
    pickle.dump(label_encoder_gender, file)

with open('pickle_files/onehot_encoder_geo.pkl', 'wb') as file:
    pickle.dump(onehot_encoder_geo, file)

with open('pickle_files/scaler.pkl', 'wb') as file:
    pickle.dump(scaler, file)

In [4]:
#Define a function to create model and try different parameters(KerasClassifier)

def create_model(neurons=32,layers=1):
    model=Sequential()
    model.add(Dense(neurons,activation='relu',input_shape=(X_train.shape[1],)))

    for _ in range(layers-1):
        model.add(Dense(neurons,activation='relu'))

    model.add(Dense(1,activation='sigmoid'))
    model.compile(optimizer='adam',loss="binary_crossentropy",metrics=['accuracy'])

    return model

In [5]:
## Create a Keras classifier
model=KerasClassifier(layers=1,neurons=32,build_fn=create_model,verbose=1)

In [6]:
# Define the grid search parameters
param_grid = {
    'neurons': [16, 32, 64, 128],
    'layers': [1, 2],
    'epochs': [50, 100]
}

In [7]:
# Perform grid search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3,verbose=1)
grid_result = grid.fit(X_train, y_train)

# Print the best parameters
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Fitting 3 folds for each of 16 candidates, totalling 48 fits
Epoch 1/50


c:\Users\Abhishek\PROJECTS\ANN_Project\.venv\Lib\site-packages\scikeras\wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
c:\Users\Abhishek\PROJECTS\ANN_Project\.venv\Lib\site-packages\keras\src\layers\core\dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7968 - loss: 0.4814
Epoch 2/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8136 - loss: 0.4354  
Epoch 3/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 964us/step - accuracy: 0.8163 - loss: 0.4225
Epoch 4/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 912us/step - accuracy: 0.8206 - loss: 0.4142
Epoch 5/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8250 - loss: 0.4074  
Epoch 6/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 811us/step - accuracy: 0.8255 - loss: 0.4013
Epoch 7/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 778us/step - accuracy: 0.8299 - loss: 0.3958
Epoch 8/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 865us/step - accuracy: 0.8324 - loss: 0.3914
Epoch 9/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 795us/step - accuracy: 0.8357 - loss: 0.3866
Epoch 10/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 786us/step - accuracy: 0.8394 - loss: 0.3822
Epoch 11/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 872us/step - accuracy: 0.8415 - loss: 0.3774
Epoch 12/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s